<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/149_Agent_02_Context_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Agent Code

In [ ]:
from typing import Dict, List, Any
import json

class Agent:
    """Base class for all agents"""
    def __init__(self, name: str):
        self.name = name

    def execute(self, task: str, context: Dict = None) -> Dict:
        """Override this method in specific agents"""
        raise NotImplementedError

class ResearchAgent(Agent):
    """Simple research agent"""
    def execute(self, task: str, context: Dict = None) -> Dict:
        # Simulate research work
        return {
            "agent": self.name,
            "result": f"Research completed for: {task}",
            "data": {"findings": ["fact1", "fact2", "fact3"]},
            "status": "success"
        }

class WriterAgent(Agent):
    """Simple writing agent"""
    def execute(self, task: str, context: Dict = None) -> Dict:
        # Use context from previous agents if available
        research_data = context.get("research_data", []) if context else []
        return {
            "agent": self.name,
            "result": f"Article written about: {task}",
            "data": {"article": f"Based on research {research_data}, here's the article..."},
            "status": "success"
        }

class BasicOrchestrator:
    """The simplest possible orchestrator"""

    def __init__(self):
        # 1. AGENT REGISTRY - catalog of available agents
        self.agents: Dict[str, Agent] = {}

        # 2. EXECUTION CONTEXT - shared state between agents
        self.context: Dict[str, Any] = {}

    def register_agent(self, agent: Agent):
        """Add an agent to our toolshed"""
        self.agents[agent.name] = agent
        print(f"Registered agent: {agent.name}")

    def execute_workflow(self, workflow: List[Dict]) -> List[Dict]:
        """
        3. WORKFLOW EXECUTION - the core orchestration logic

        workflow format: [
            {"agent": "research", "task": "Find info about AI"},
            {"agent": "writer", "task": "Write article about AI"}
        ]
        """
        results = []

        for step in workflow:
            agent_name = step["agent"]
            task = step["task"]

            # Get the agent from our registry
            if agent_name not in self.agents:
                results.append({
                    "error": f"Agent '{agent_name}' not found",
                    "status": "failed"
                })
                break

            agent = self.agents[agent_name]

            # Execute the agent with current context
            try:
                result = agent.execute(task, self.context)
                results.append(result)

                # 4. CONTEXT MANAGEMENT - update shared state
                # Pass results to next agents
                if result["status"] == "success":
                    if "data" in result:
                        key = f"{agent_name}_data"
                        self.context[key] = result["data"]

                print(f"✓ {agent_name}: {result['result']}")

            except Exception as e:
                error_result = {
                    "agent": agent_name,
                    "error": str(e),
                    "status": "failed"
                }
                results.append(error_result)
                print(f"✗ {agent_name}: {str(e)}")
                break  # Stop on first failure

        return results

# Example usage
def main():
    # Create orchestrator
    orchestrator = BasicOrchestrator()

    # Register agents (build our toolshed)
    orchestrator.register_agent(ResearchAgent("research"))
    orchestrator.register_agent(WriterAgent("writer"))

    # Define a simple workflow
    workflow = [
        {"agent": "research", "task": "Find information about AI orchestration"},
        {"agent": "writer", "task": "Write an article about AI orchestration"}
    ]

    # Execute workflow
    print("\n--- Executing Workflow ---")
    results = orchestrator.execute_workflow(workflow)

    # Show results
    print("\n--- Results ---")
    for i, result in enumerate(results):
        print(f"Step {i+1}: {json.dumps(result, indent=2)}")

if __name__ == "__main__":
    main()


Here's the most basic orchestrator that demonstrates the core concepts:This bare-bones orchestrator demonstrates the **4 critical components** that every orchestrator must have:

## **1. Agent Registry**
- A catalog of available agents (your "toolshed")
- Allows dynamic discovery and selection of agents
- Makes the system modular and extensible

## **2. Workflow Execution Engine**
- The core logic that runs agents in sequence
- Handles the "what happens next" decisions
- This is where orchestration actually happens

## **3. Context Management**
- Shared state that flows between agents
- Allows agents to build on each other's work
- Critical for multi-step workflows

## **4. Error Handling**
- What happens when an agent fails
- Determines if workflow continues or stops
- Essential for reliability

**Why this is the foundation:**
- **Simple**: Only ~100 lines but contains all core concepts
- **Extensible**: Easy to add new agents without changing orchestrator
- **Testable**: Each component can be tested independently
- **Understandable**: Clear separation of concerns

**What's missing (we'll add later):**
- Parallel execution
- Conditional logic
- Agent selection strategies  
- Sophisticated error recovery
- State persistence
- Monitoring/observability

Try running this code! You can easily add new agents by inheriting from the `Agent` class and registering them. The workflow format is dead simple but powerful.



Perfect! **Context Management** is where workflows become truly intelligent - it's the "memory system" that allows steps to build on each other's work and enables complex reasoning across the entire workflow.## **What You Should Focus On & Learn:**

### **🧠 Context as the "Workflow Brain"**

Context Management is what transforms a simple sequence of steps into an **intelligent reasoning system**. It enables:

1. **Memory**: "What did we learn in previous steps?"
2. **Communication**: "How do steps share information?"
3. **Intelligence**: "Should we branch differently based on what we found?"
4. **Efficiency**: "Can we reuse previous computations?"

### **🔑 Critical Concepts to Master:**

#### **1. Context Scopes** - The Visibility System
```python
ContextScope.STEP      # Only this step can see it
ContextScope.WORKFLOW  # All steps in workflow can see it  
ContextScope.SESSION   # All workflows in user session
ContextScope.GLOBAL    # System-wide data
```
**Why this matters:** Controls data visibility and prevents information leakage between unrelated workflows.

#### **2. Context Flow** - The Information Pipeline
```python
# Step 1 produces data
research_result = {"market_size": 1.2B, "confidence": 0.9}

# Step 2 automatically gets access to Step 1's results
step2_context = {
    "research_result": research_result,  # Direct access
    "dependencies": {"research": research_result},  # Structured access
    "confidence_score": 0.9  # Named outputs for easy reference
}
```
**Why this matters:** This is how steps "build on each other" instead of working in isolation.

#### **3. Conditional Logic** - Dynamic Workflow Branching
```python
# Workflow can adapt based on intermediate results
if context.evaluate_condition("confidence_score >= 0.8"):
    execute_step("detailed_analysis")
else:
    execute_step("gather_more_data")
```
**Why this matters:** Enables workflows that adapt intelligently to what they discover.

#### **4. Memory Management** - Intelligent Resource Usage
```python
# Automatic cleanup of expired data
# LRU eviction when memory gets full
# Size tracking to prevent memory bloat
```
**Why this matters:** Prevents workflows from consuming unlimited memory as they run.

## **🎯 The Key Intelligence Features:**

### **A. Smart Context Building**
Before each step executes, the context manager:
1. **Gathers relevant data** from previous steps
2. **Provides dependency results** in easily accessible format
3. **Includes workflow metadata** for agent awareness
4. **Manages memory efficiently** to prevent bloat

### **B. Automatic Result Processing**
After each step completes, the context manager:
1. **Stores complete results** for debugging/auditing
2. **Extracts key outputs** for easy access by later steps
3. **Creates named references** for intuitive data access
4. **Tracks execution metadata** for performance analysis

### **C. Dynamic Condition Evaluation**
The context manager enables:
1. **Path-based conditions**: `"research_result.confidence > 0.8"`
2. **Boolean flags**: `"user_approved"`
3. **Complex logic**: `"threat_level < 0.5 AND budget > 100000"`

## **🚀 Why This Creates Emergent Intelligence:**

### **From Simple Steps to Complex Reasoning:**
```python
# Without context: Isolated steps
step1: research_topic()      # → result thrown away
step2: analyze_data()        # → starts from scratch
step3: write_report()        # → no awareness of findings

# With context: Intelligent flow
step1: research_topic()      # → stores findings in context
step2: analyze_data()        # → uses research findings, stores insights  
step3: write_report()        # → synthesizes all previous work
```

### **Dynamic Adaptation:**
```python
# Workflow can change behavior based on what it discovers
if research_confidence > 0.8:
    # High confidence: proceed with advanced analysis
    workflow.add_step("deep_analysis")
else:
    # Low confidence: gather more data first
    workflow.add_step("additional_research")
```

## **🔧 The Technical Beauty:**

### **Intelligent Memory Management:**
- **TTL (Time To Live)**: Data expires when no longer needed
- **LRU Eviction**: Removes least-used data when memory is full  
- **Size Tracking**: Prevents any single piece of data from dominating memory
- **Access Patterns**: Optimizes based on how data is actually used

### **Multi-Level Context Hierarchy:**
```python
# Agent sees consolidated view:
{
    "global_user_preferences": {...},     # System-wide settings
    "session_data": {...},                # User session context
    "workflow_config": {...},             # Workflow-specific data
    "research_result": {...},             # Previous step output
    "dependencies": {"step1": {...}},     # Structured dependency access
    "_workflow_meta": {...}               # System information
}
```

## **🎪 Real-World Impact:**

### **Example: Market Analysis Workflow**
1. **Research step** finds market size = $1.2B, confidence = 90%
2. **Context manager** stores this data with metadata
3. **Analysis step** gets research data, determines threat level = 40%
4. **Context manager** evaluates: `"confidence >= 0.8 AND threat_level < 0.5"`
5. **Strategy step** executes "aggressive expansion" branch instead of "cautious approach"

**The workflow literally becomes smarter** as it progresses!

## **🎯 Key Learning Points:**

1. **Context enables workflows to "remember"** what they've learned
2. **Scoped visibility** prevents data leakage between workflows
3. **Automatic memory management** keeps the system efficient
4. **Conditional logic** enables dynamic workflow adaptation
5. **Rich metadata** provides debugging and optimization insights



Context Management is where your orchestrator becomes **truly intelligent** - it's the difference between a task runner and a reasoning system! 🧠✨

In [ ]:
from typing import Dict, List, Any, Optional, Union
from dataclasses import dataclass, field
from enum import Enum
import json
import time
from copy import deepcopy

class ContextScope(Enum):
    """Different levels of context visibility"""
    STEP = "step"           # Only visible to current step
    WORKFLOW = "workflow"   # Visible to entire workflow
    GLOBAL = "global"       # Visible across all workflows
    SESSION = "session"     # Visible for user session

@dataclass
class ContextEntry:
    """Individual piece of context data with metadata"""
    key: str                    # Unique identifier
    value: Any                  # The actual data
    scope: ContextScope         # Visibility level
    created_at: float           # When it was created
    created_by: str             # Which step/agent created it

    # Data management
    ttl: Optional[int] = None   # Time to live in seconds
    size_bytes: int = 0         # Memory usage tracking
    access_count: int = 0       # Usage tracking
    last_accessed: float = field(default_factory=time.time)

    # Metadata
    data_type: str = "unknown"  # Type hint for agents
    description: str = ""       # Human-readable description
    sensitive: bool = False     # Contains sensitive data?

class ContextManager:
    """
    CRITICAL COMPONENT: The workflow's memory and communication system

    This is what enables:
    1. Steps to access results from previous steps
    2. Complex reasoning across multiple steps
    3. Dynamic workflow adaptation based on intermediate results
    4. Efficient data sharing without redundant computation
    """

    def __init__(self, max_memory_mb: int = 100):
        # Core storage - hierarchical by scope
        self._contexts: Dict[ContextScope, Dict[str, ContextEntry]] = {
            scope: {} for scope in ContextScope
        }

        # Memory management
        self.max_memory_bytes = max_memory_mb * 1024 * 1024
        self.current_memory_usage = 0

        # Access tracking for optimization
        self._access_patterns: Dict[str, List[float]] = {}

        # Context history for debugging
        self._context_history: List[Dict] = []

    def set_context(self, key: str, value: Any, scope: ContextScope = ContextScope.WORKFLOW,
                   created_by: str = "system", **metadata) -> bool:
        """
        CORE METHOD: Store data in context with intelligent management
        """
        try:
            # Calculate memory usage
            size_bytes = self._calculate_size(value)

            # Check memory limits
            if not self._check_memory_capacity(size_bytes):
                self._cleanup_expired_context()
                if not self._check_memory_capacity(size_bytes):
                    self._evict_least_used_context(size_bytes)

            # Create context entry
            entry = ContextEntry(
                key=key,
                value=value,
                scope=scope,
                created_at=time.time(),
                created_by=created_by,
                size_bytes=size_bytes,
                data_type=type(value).__name__,
                description=metadata.get('description', ''),
                sensitive=metadata.get('sensitive', False),
                ttl=metadata.get('ttl')
            )

            # Store the entry
            self._contexts[scope][key] = entry
            self.current_memory_usage += size_bytes

            # Record the change
            self._record_context_change("set", key, scope, created_by)

            print(f"📝 Context set: {key} ({scope.value}) by {created_by}")
            return True

        except Exception as e:
            print(f"❌ Failed to set context {key}: {e}")
            return False

    def get_context(self, key: str, requestor: str = "system",
                   scope_priority: List[ContextScope] = None) -> Any:
        """
        INTELLIGENT RETRIEVAL: Get context with scope-aware lookup
        """
        if scope_priority is None:
            # Default priority: most specific to most general
            scope_priority = [ContextScope.STEP, ContextScope.WORKFLOW,
                            ContextScope.SESSION, ContextScope.GLOBAL]

        for scope in scope_priority:
            if key in self._contexts[scope]:
                entry = self._contexts[scope][key]

                # Check if expired
                if self._is_expired(entry):
                    self._remove_context(key, scope)
                    continue

                # Update access tracking
                entry.access_count += 1
                entry.last_accessed = time.time()
                self._track_access(key, requestor)

                print(f"📖 Context retrieved: {key} ({scope.value}) by {requestor}")
                return entry.value

        print(f"🔍 Context not found: {key} (requested by {requestor})")
        return None

    def build_step_context(self, workflow_id: str, step_id: str,
                          dependencies: List[str] = None) -> Dict[str, Any]:
        """
        STEP PREPARATION: Build complete context for step execution
        This is what gets passed to agents when they execute
        """
        context = {}

        # 1. Include global context (system-wide data)
        for key, entry in self._contexts[ContextScope.GLOBAL].items():
            if not self._is_expired(entry):
                context[f"global_{key}"] = entry.value

        # 2. Include workflow context (shared across steps)
        for key, entry in self._contexts[ContextScope.WORKFLOW].items():
            if not self._is_expired(entry):
                context[key] = entry.value

        # 3. Include dependency results (outputs from previous steps)
        if dependencies:
            context["dependencies"] = {}
            for dep_step_id in dependencies:
                dep_result = self.get_context(f"step_{dep_step_id}_result")
                if dep_result:
                    context["dependencies"][dep_step_id] = dep_result
                    # Also add direct access for convenience
                    context[f"{dep_step_id}_result"] = dep_result

        # 4. Add workflow metadata
        context["_workflow_meta"] = {
            "workflow_id": workflow_id,
            "current_step": step_id,
            "timestamp": time.time(),
            "available_data": list(context.keys())
        }

        # 5. Add memory usage info for agents
        context["_system_info"] = {
            "memory_usage_mb": self.current_memory_usage / (1024 * 1024),
            "context_size": len(context),
            "high_memory_usage": self.current_memory_usage > (self.max_memory_bytes * 0.8)
        }

        print(f"🧠 Built context for {step_id}: {len(context)} entries")
        return context

    def store_step_result(self, workflow_id: str, step_id: str, result: Dict,
                         agent_name: str) -> bool:
        """
        RESULT PROCESSING: Store step results in context intelligently
        """
        try:
            # Store the complete result
            result_key = f"step_{step_id}_result"
            self.set_context(
                result_key,
                result,
                ContextScope.WORKFLOW,
                created_by=f"{agent_name}@{step_id}",
                description=f"Complete result from step {step_id}",
                ttl=3600  # Results expire after 1 hour
            )

            # Extract and store key outputs for easy access
            if "data" in result:
                data_key = f"{step_id}_data"
                self.set_context(
                    data_key,
                    result["data"],
                    ContextScope.WORKFLOW,
                    created_by=f"{agent_name}@{step_id}",
                    description=f"Data output from step {step_id}"
                )

            # Store named outputs if provided
            if "outputs" in result:
                for output_name, output_value in result["outputs"].items():
                    self.set_context(
                        output_name,
                        output_value,
                        ContextScope.WORKFLOW,
                        created_by=f"{agent_name}@{step_id}",
                        description=f"Named output '{output_name}' from step {step_id}"
                    )

            # Store execution metadata
            exec_meta_key = f"step_{step_id}_meta"
            execution_metadata = {
                "agent": agent_name,
                "execution_time": result.get("execution_time", 0),
                "status": result.get("status", "unknown"),
                "timestamp": time.time(),
                "step_id": step_id,
                "workflow_id": workflow_id
            }
            self.set_context(
                exec_meta_key,
                execution_metadata,
                ContextScope.WORKFLOW,
                created_by="system",
                description=f"Execution metadata for step {step_id}"
            )

            return True

        except Exception as e:
            print(f"❌ Failed to store result for {step_id}: {e}")
            return False

    def create_context_summary(self, workflow_id: str) -> Dict:
        """
        WORKFLOW INTELLIGENCE: Create summary of available context
        This helps agents understand what data they have access to
        """
        summary = {
            "workflow_context": {},
            "step_results": {},
            "global_data": {},
            "memory_usage": {
                "current_mb": self.current_memory_usage / (1024 * 1024),
                "max_mb": self.max_memory_bytes / (1024 * 1024),
                "utilization": (self.current_memory_usage / self.max_memory_bytes) * 100
            }
        }

        # Categorize context by type
        for scope, context_dict in self._contexts.items():
            for key, entry in context_dict.items():
                if self._is_expired(entry):
                    continue

                entry_info = {
                    "data_type": entry.data_type,
                    "size_mb": entry.size_bytes / (1024 * 1024),
                    "created_by": entry.created_by,
                    "created_at": entry.created_at,
                    "access_count": entry.access_count,
                    "description": entry.description
                }

                if scope == ContextScope.WORKFLOW:
                    if key.startswith("step_") and key.endswith("_result"):
                        summary["step_results"][key] = entry_info
                    else:
                        summary["workflow_context"][key] = entry_info
                elif scope == ContextScope.GLOBAL:
                    summary["global_data"][key] = entry_info

        return summary

    def evaluate_context_condition(self, condition: str, workflow_id: str) -> bool:
        """
        CONDITIONAL LOGIC: Evaluate conditions based on context
        This enables dynamic workflow branching
        """
        try:
            # Build evaluation context
            eval_context = {}

            # Add workflow context for evaluation
            for key, entry in self._contexts[ContextScope.WORKFLOW].items():
                if not self._is_expired(entry):
                    eval_context[key] = entry.value

            # Simple condition evaluation (could be much more sophisticated)
            # Examples:
            # "step_analysis_result.confidence > 0.8"
            # "research_data.source_count >= 5"
            # "user_approval == true"

            # For demo, use a simple path-based evaluation
            if "." in condition:
                # Handle dotted path like "step_result.confidence > 0.8"
                parts = condition.split()
                if len(parts) == 3:  # "path operator value"
                    path, operator, value_str = parts

                    # Navigate the path
                    path_parts = path.split(".")
                    current = eval_context
                    for part in path_parts:
                        if isinstance(current, dict) and part in current:
                            current = current[part]
                        else:
                            return False

                    # Evaluate condition
                    try:
                        expected_value = float(value_str) if value_str.replace(".", "").isdigit() else value_str.strip('"\'')
                        if operator == ">":
                            return float(current) > float(expected_value)
                        elif operator == "<":
                            return float(current) < float(expected_value)
                        elif operator == ">=":
                            return float(current) >= float(expected_value)
                        elif operator == "<=":
                            return float(current) <= float(expected_value)
                        elif operator == "==":
                            return str(current) == str(expected_value)
                        elif operator == "!=":
                            return str(current) != str(expected_value)
                    except:
                        return False
            else:
                # Simple boolean check
                return bool(eval_context.get(condition, False))

            return False

        except Exception as e:
            print(f"❌ Failed to evaluate condition '{condition}': {e}")
            return False

    def _calculate_size(self, value: Any) -> int:
        """Estimate memory usage of a value"""
        try:
            return len(json.dumps(value, default=str).encode('utf-8'))
        except:
            return len(str(value).encode('utf-8'))

    def _check_memory_capacity(self, required_bytes: int) -> bool:
        """Check if we have enough memory for new data"""
        return (self.current_memory_usage + required_bytes) <= self.max_memory_bytes

    def _cleanup_expired_context(self):
        """Remove expired context entries"""
        for scope in self._contexts:
            expired_keys = []
            for key, entry in self._contexts[scope].items():
                if self._is_expired(entry):
                    expired_keys.append(key)

            for key in expired_keys:
                self._remove_context(key, scope)

    def _is_expired(self, entry: ContextEntry) -> bool:
        """Check if context entry has expired"""
        if entry.ttl is None:
            return False
        return (time.time() - entry.created_at) > entry.ttl

    def _evict_least_used_context(self, required_bytes: int):
        """Remove least-used context to free memory"""
        # Collect all entries with usage info
        all_entries = []
        for scope in self._contexts:
            for key, entry in self._contexts[scope].items():
                all_entries.append((key, scope, entry))

        # Sort by access count and last access time (LRU-like)
        all_entries.sort(key=lambda x: (x[2].access_count, x[2].last_accessed))

        # Remove entries until we have enough space
        freed_bytes = 0
        for key, scope, entry in all_entries:
            if freed_bytes >= required_bytes:
                break

            # Don't evict very recently created entries
            if (time.time() - entry.created_at) < 60:  # Don't evict entries < 1 minute old
                continue

            freed_bytes += entry.size_bytes
            self._remove_context(key, scope)
            print(f"🗑️ Evicted context: {key} ({scope.value}) to free memory")

    def _remove_context(self, key: str, scope: ContextScope):
        """Remove context entry and update memory tracking"""
        if key in self._contexts[scope]:
            entry = self._contexts[scope][key]
            self.current_memory_usage -= entry.size_bytes
            del self._contexts[scope][key]
            self._record_context_change("remove", key, scope, "system")

    def _track_access(self, key: str, requestor: str):
        """Track access patterns for optimization"""
        if key not in self._access_patterns:
            self._access_patterns[key] = []
        self._access_patterns[key].append(time.time())

        # Keep only recent access history
        cutoff_time = time.time() - 3600  # Last hour
        self._access_patterns[key] = [t for t in self._access_patterns[key] if t > cutoff_time]

    def _record_context_change(self, action: str, key: str, scope: ContextScope, actor: str):
        """Record context changes for debugging and auditing"""
        self._context_history.append({
            "action": action,
            "key": key,
            "scope": scope.value,
            "actor": actor,
            "timestamp": time.time(),
            "memory_usage_mb": self.current_memory_usage / (1024 * 1024)
        })

        # Keep only recent history
        if len(self._context_history) > 1000:
            self._context_history = self._context_history[-500:]  # Keep last 500 changes

# Example usage showing context flow in a real workflow
def demo_context_management():
    """Demonstrate how context enables intelligent workflows"""

    context_manager = ContextManager(max_memory_mb=50)

    print("=== Context Management Demo ===")

    # Simulate workflow execution with context flow
    workflow_id = "market_analysis_001"

    # Step 1: Research phase
    print("\n--- Step 1: Market Research ---")
    research_result = {
        "status": "success",
        "data": {
            "market_size": 1200000000,
            "growth_rate": 0.15,
            "key_players": ["Company A", "Company B", "Company C"],
            "trends": ["AI adoption", "Cloud migration", "Sustainability"]
        },
        "outputs": {
            "market_analysis": "Market shows strong growth potential",
            "confidence_score": 0.9
        },
        "execution_time": 45.2
    }

    context_manager.store_step_result(workflow_id, "research", research_result, "research_agent")

    # Step 2: Competitive analysis (depends on research)
    print("\n--- Step 2: Competitive Analysis ---")
    step2_context = context_manager.build_step_context(
        workflow_id, "competitive_analysis", dependencies=["research"]
    )

    print("Context available to competitive analysis:")
    for key in step2_context.keys():
        if not key.startswith("_"):
            print(f"  - {key}: {type(step2_context[key]).__name__}")

    # Competitive analysis uses research data
    competitive_result = {
        "status": "success",
        "data": {
            "competitive_landscape": "Fragmented market with opportunities",
            "market_share_analysis": {"Company A": 0.3, "Company B": 0.25, "Others": 0.45},
            "threat_assessment": "Medium threat level"
        },
        "outputs": {
            "competitive_summary": "Good positioning opportunity exists",
            "threat_level": 0.4
        },
        "execution_time": 32.1
    }

    context_manager.store_step_result(workflow_id, "competitive_analysis", competitive_result, "analysis_agent")

    # Step 3: Strategic recommendation (conditional based on results)
    print("\n--- Step 3: Strategic Recommendation ---")

    # Check if conditions are met for aggressive strategy
    aggressive_condition = "competitive_analysis_data.threat_assessment == 'Medium threat level'"
    should_be_aggressive = context_manager.evaluate_context_condition(aggressive_condition, workflow_id)
    print(f"Should use aggressive strategy: {should_be_aggressive}")

    # Check confidence threshold
    confidence_condition = "confidence_score >= 0.8"
    high_confidence = context_manager.evaluate_context_condition(confidence_condition, workflow_id)
    print(f"High confidence in analysis: {high_confidence}")

    # Build context for strategy step
    strategy_context = context_manager.build_step_context(
        workflow_id, "strategy", dependencies=["research", "competitive_analysis"]
    )

    print(f"\nFinal strategy context contains {len(strategy_context)} elements")

    # Show context summary
    print("\n--- Context Summary ---")
    summary = context_manager.create_context_summary(workflow_id)
    print(f"Workflow context entries: {len(summary['workflow_context'])}")
    print(f"Step results stored: {len(summary['step_results'])}")
    print(f"Memory usage: {summary['memory_usage']['current_mb']:.2f} MB")

    print("\nStep results available:")
    for key, info in summary['step_results'].items():
        print(f"  - {key}: {info['data_type']} by {info['created_by']}")

if __name__ == "__main__":
    demo_context_management()